In [26]:
import pandas as pd
import numpy as np

In [27]:
df = pd.read_excel("CSAT_Cleaned.xlsx", sheet_name="CSAT answers") 

In [28]:
theme_dict = {
    "Access_Wait": [
        "wait ", "waiting", "on hold", "hold for", "queue", "in a queue",
        "long time", "long wait", "no one answered", "no one picks up",
        "couldn't get through", "cant get through", "get through to",
        "line was busy", "lines are busy", "call dropped", "cut off",
        "disconnected"
    ],
    
    "Multiple_Contacts": [
        "called again", "call again", "called back", "call back", "ring back",
        "phoned again", "several times", "many times", "multiple times",
        "more than once", "over and over", "keep calling", "kept calling",
        "chasing", "had to call back", "had to ring back"
    ],

    "Staff_Positive": [
        "helpful", "very helpful", "so helpful", "polite", "very polite",
        "friendly", "professional", "kind", "patient", "respectful", "lovely",
        "excellent service", "great service", "brilliant", "amazing",
        "above and beyond", "listened", "explained everything",
        "explained clearly", "clear explanation"
    ],

    "Staff_Negative": [
        "rude", "unhelpful", "not helpful", "couldn't help",
        "wouldn't help", "didn't care", "not bothered",
        "couldn't be bothered", "didn't listen", "ignored", "hung up",
        "put the phone down", "bad attitude", "poor attitude",
        "unprofessional", "useless", "appalling", "disgusting",
        "unacceptable"
    ],

    "Comms_Updates": [
        "no update", "no updates", "wasn't updated", "was not updated",
        "keep me updated", "kept me updated", "kept informed",
        "wasn't informed", "no communication", "lack of communication",
        "nobody told", "no one told", "nobody called back",
        "no one called back", "didn't call back", "promised to call back",
        "never called back", "waiting for a call back", "waiting for callback",
        "emailed", "sent me an email", "letter", "text", "sms", "message me"
    ],

    "Resolution_Fix": [
        "not resolved", "unresolved", "still not resolved", "still not sorted",
        "issue remains", "problem remains", "problem came back",
        "came back again", "not fixed", "never fixed", "still leaking",
        "leak", "leaking", "leak allowance", "repair", "repaired",
        "fix", "fixed", "engineer", "appointment", "booked an appointment",
        "cancelled the appointment", "no one turned up", "didn't turn up",
        "no show", "site visit", "job was closed", "case closed",
        "temporary fix", "taking too long", "took too long", "waited"
    ],

    "Billing_Meter": [
        "bill", "billing", "high bill", "large bill", "bigger bill",
        "overcharged", "over charge", "charge", "charges", "credit",
        "in credit", "debit", "payment", "payment plan", "direct debit",
        "dd", "tariff", "rate", "meter", "meter reading", "reading",
        "usage", "consumption", "smart meter"
    ],

    "Digital_Online": [
        "online", "website", "web site", "app", "mobile app", "logged in",
        "login", "account", "my account", "form", "online form",
        "application form", "leak allowance form", "webchat", "live chat",
        "chatbot", "bot"
    ]
}


In [29]:
# Clean comments: lowercase + fill blanks
df["comment_clean"] = (
    df["comments_verb"]
    .astype(str)
    .str.lower()
    .fillna("")
)


In [30]:
def tag_themes(text, theme_dict):
    """Return a dictionary of theme flags for one text row."""
    flags = {}
    for theme, keywords in theme_dict.items():
        flags[theme] = int(any(kw in text for kw in keywords))
    return flags

# Expand tags into columns
tagged = df["comment_clean"].apply(lambda x: pd.Series(tag_themes(x, theme_dict)))

df = pd.concat([df, tagged], axis=1)

In [31]:
df.head()

,csat_id,Survey_Date,Journey,Sub_Area,Area,Department,Business_Unit,Channel,Score,cid,...,comment_lower,comment_clean,Access_Wait,Multiple_Contacts,Staff_Positive,Staff_Negative,Comms_Updates,Resolution_Fix,Billing_Meter,Digital_Online
0,1,2025-05-01,Invoice/statement - query,Retail,NaN,NaN,Revenue,T,0,10,...,i feel i was bullied into accepting your price...,i feel i was bullied into accepting your price...,0,0,0,0,0,0,1,1
1,2,2025-05-02,Payment scheme/instalment plan - set up,Retail,Thames Water,Thames Water - Case Management,Revenue,T,10,10,...,NaN,nan,0,0,0,0,0,0,0,0
2,3,2025-05-03,Payment Made,Retail,Tech Mahindra,Tech Mahindra - First Contact,Revenue,T,10,10,...,excellent help.,excellent help.,0,0,0,0,0,0,0,0
3,4,2025-05-03,Request a meter - query,Retail,Thames Water,Thames Water - First Contact,Revenue,T,10,10,...,helpfulness of lady i spoke to. she listened t...,helpfulness of lady i spoke to. she listened t...,0,0,1,0,0,0,0,0
4,5,2025-05-04,High Bill -Suspected Leak,Retail,Thames Water,Thames Water - First Contact,Revenue,T,10,10,...,your adviser was so helpful and understanding....,your adviser was so helpful and understanding....,0,0,1,0,0,0,0,0


In [32]:
summary = (
    df.groupby("cid")[list(theme_dict.keys())]
    .mean()
    .reset_index()
)

In [33]:
summary

,cid,Access_Wait,Multiple_Contacts,Staff_Positive,Staff_Negative,Comms_Updates,Resolution_Fix,Billing_Meter,Digital_Online
0,10,0.031217,0.014012,0.335441,0.022093,0.009711,0.049726,0.112357,0.099127
1,42,0.016750,0.008375,0.103853,0.015075,0.008375,0.083752,0.031826,0.067002
2,43,0.029248,0.016713,0.097493,0.029248,0.018106,0.080780,0.178273,0.126741
3,44,0.037500,0.013657,0.197454,0.023611,0.019676,0.127315,0.111574,0.115278


In [34]:
negative = df[df["Sentiment_Score"] < 2]

neg_summary = (
    negative.groupby("cid")[list(theme_dict.keys())]
    .mean()
    .reset_index()
)
neg_summary

,cid,Access_Wait,Multiple_Contacts,Staff_Positive,Staff_Negative,Comms_Updates,Resolution_Fix,Billing_Meter,Digital_Online
0,10,0.109258,0.064502,0.097411,0.130759,0.031593,0.135586,0.332163,0.216323
1,42,0.057692,0.000000,0.076923,0.134615,0.057692,0.288462,0.173077,0.230769
2,43,0.051724,0.063218,0.074713,0.114943,0.051724,0.143678,0.402299,0.264368
3,44,0.099881,0.045184,0.059453,0.105826,0.051130,0.237812,0.297265,0.242568


In [35]:
df.to_excel("CSAT_Cleaned_Tagged.xlsx",sheet_name="CSAT answers", index=False)

In [36]:
theme_cols = list(theme_dict.keys())

df["No_Theme"] = (df[theme_cols].sum(axis=1) == 0).astype(int)


In [37]:
df["No_Theme"].value_counts()

No_Theme
1    11263
0     9716
Name: count, dtype: int64